In [1]:
import numpy as np
import matplotlib.pyplot as plt
# from sklearn.datasets import fetch_openml # to download mnist data
import cv2 as cv

# # add the path to my packages to system paths so they can be imported
# import sys
# sys.path.append('/home/yasamanparhizkar/Documents/yorku/01_thesis/code/my_packages')
# # sys.path.append('F:\MAScThesis\code\my_packages')
# # sys.path.append('/home/yasamanparhizkar/Documents/thesis/code/my_packages')

# import sift_on_mnist

# Setup the mnist-SIFT data

In [2]:
data_path = '../../data/fe_exp/mnist-org/'
data_sz = 14780 # all available images on my personal computer right now
img_sz = (28, 28)

data_y = np.loadtxt(data_path+'lbls.txt')
data_y = data_y[:data_sz]

In [3]:
data_x = np.array([np.loadtxt(data_path+'fv_'+str(0)+'.txt')])
for vect_i in range(1, data_sz):
    data_x = np.append(data_x, [np.loadtxt(data_path+'fv_'+str(vect_i)+'.txt')], axis=0)

In [4]:
print(data_x.shape)
print(data_y.shape)

(14780, 784)
(14780,)


In [5]:
# convert train and validation images from 1D to 2D with 3 channels of [0,255] pixel values
data_x_sift = ((data_x.reshape((data_x.shape[0], img_sz[0], img_sz[1], 1)))*256).astype('uint8')
data_x_sift = np.broadcast_to(data_x_sift, (data_x.shape[0], img_sz[0], img_sz[1], 3))

In [6]:
# SIFT default parameter values are:
nfeatures = 0
nOctaveLayers = 3
contrastThreshold = 0.04
edgeThreshold = 10
sigma = 1.6

sift = cv.SIFT_create(nfeatures, nOctaveLayers, contrastThreshold, edgeThreshold, sigma)

# extract SIFT descriptors for all training and validation images
gray = np.array([cv.cvtColor(data_x_sift[0],cv.COLOR_BGR2GRAY)])
for i in range(1, data_x_sift.shape[0]):
    gray = np.append(gray, [cv.cvtColor(data_x_sift[i],cv.COLOR_BGR2GRAY)], axis=0)
kp = []
des = []
for i in range(gray.shape[0]):
    _kp, _des = sift.detectAndCompute(gray[i],None)
    kp.append(_kp)
    des.append(_des)

In [7]:
# discard all images that have less than 2 keypoints
num_keypoints = [len(kp[i]) for i in range(len(kp))]
to_discard_ind = [i for i in range(len(num_keypoints)) if (num_keypoints[i]==0 or num_keypoints[i]==1)]
x_sift_mod = np.delete(data_x_sift, to_discard_ind, axis=0)
y_mod = np.delete(data_y, to_discard_ind, axis=0)
gray_mod = np.delete(gray, to_discard_ind, axis=0)
kp_mod = [kp[i] for i in range(len(kp)) if i not in to_discard_ind]
des_mod = [des[i] for i in range(len(des)) if i not in to_discard_ind]

# find new number of train & val images
data_sz_mod = data_sz - len(to_discard_ind)

In [8]:
# select only two keypoints in all images with more than 2 keypoints
for i in range(len(kp_mod)): # for each training or validation image
    # calculate keypoint's distance from the top-left and bottom-right corners
    left_dists = [(kp_mod[i][j], np.sum(np.array(kp_mod[i][j].pt)**2), des_mod[i][j]) for j in range(len(kp_mod[i]))]
    right_dists = [(kp_mod[i][j], np.sum(np.array([img_sz[0]-kp_mod[i][j].pt[0], img_sz[1]-kp_mod[i][j].pt[1]])**2), \
                    des_mod[i][j]) for j in range(len(kp_mod[i]))]

    # select closest keypoints to the top-left and bottom-right corners, discard all other keypoints
    kp_mod[i] = (min(left_dists, key=lambda x:x[1])[0], min(right_dists, key=lambda x:x[1])[0])
    des_mod[i] = np.array([min(left_dists, key=lambda x:x[1])[2], min(right_dists, key=lambda x:x[1])[2]])

    # concatenate two descriptors
    des_mod[i] = des_mod[i].reshape(-1)
des_mod = np.array(des_mod)

In [9]:
# Normalize descriptors by method 2: double normalization
# step 1 - feature-wise: subtract mean and divide by standard deviation of each feature.
des_mod_nrm = des_mod.copy()
des_mod_mean = np.mean(des_mod_nrm, axis=0, keepdims=True)
des_mod_std = np.std(des_mod_nrm, axis=0, keepdims=True)
des_mod_nrm = des_mod_nrm - des_mod_mean
des_mod_nrm = des_mod_nrm / (des_mod_std + 0.01)
# step 2 - smaple-wise: normalize l2-norm of each vector to 1.
des_mod_norm = np.linalg.norm(des_mod_nrm, axis=1, keepdims=True)
des_mod_nrm = des_mod_nrm / (des_mod_norm + 0.01)

In [10]:
save_path = '../../data/fe_exp/mnist-sift/'
for vect_i in range(data_sz_mod):
    np.savetxt(save_path+'fv_'+str(vect_i)+'.csv', des_mod_nrm[vect_i])
    
np.savetxt(save_path+'lbls.csv', y_mod*2-1)